# Low Frequency Receiver (LFR)

<div>
<center>
<img src="resources/solar-orbiter-landscape-image-astrium.jpg" width="800" align="center">
</center>
</div>

<div>
<center>
<img src="resources/logo_LPP.png" width="200" align="left">
</center>
</div>

<p align="center">
  <b>Alexis Jeandet &lt;alexis.jeandet@lpp.polytechnique.fr&gt;:</b><br>
  <a href="https://github.com/jeandet">Github:jeandet</a> |
  <a href="https://gitter.im/jeandet">Gitter:jeandet</a> |
  <a href="https://freenode.net/">IRC@freenode:jeandet</a>
  <br>
</p>


# Laboratory of Plasma Physics (LPP)


# Laboratory of Plasma Physics (LPP)

- Since 2009, fusion between LPTP and CETP
- Around 110+ people 
- 3 Scientific teams, [Fusion, Cold, Space] Plasmas 

# Laboratory of Plasma Physics (LPP)


- Plasmas are used in many daily applications, TVs, lights, energy production, air cleaning, IC manufacturing...  

- On the cosmic scale, 99 % of observed matter is ionized and therefore in the plasma state.


# Laboratory of Plasma Physics (LPP)

- Natural Plasmas are mostly in space and need in-situ measurements such as:
 - particles (**particles detectors**, **spectrometers**) 
 - E-field (Electric antenna)
 - B-field (**Search-coils**, Flux-gates)

# Solar Orbiter

- Sun observatory and Solar wind in-situ measurements
- Launch date: end of 2020
- Orbit 0.3 -> 0.8 AU
- Mass total/payload: 1800kg/190kg
- **Power 1100W**
- **Data downlink ~150 kbps**

<div>
<center>
<img src="resources/Solar_Orbiter_payload_annotated_625.jpg" width="800" align="center">
</center>
</div>


# Solar Orbiter

Small amount of telemetry implies heavy onboard computation.

# Radio Plasma Waves instrument (RPW)

- Focuses on waves( **E field** & **B field** ) measurements from **~0Hz** to **16MHz**
- Composed of **BIAS**, **TNR-HFR**, **TDS**, **DPU**, **LVPS-PDU**, **SCM**, **LFR**
- ~10kb/s average telemetry
- ~5.5kg
- ~13W


# Radio Plasma Waves instrument (RPW)

<div>
<center>
<img src="resources/principe_RPW.png" width="800" align="center">
</center>
</div>


# Low Frequency Receiver (LFR)

- Produces waveforms at 24576Hz, 4096Hz, 256Hz and 16Hz (F[0,1,2,3])

- Computes 256 points FFTs on F0 to F2 continuously (5 components 2E and 3B)

- Computes and averages Spectral Matrices continuously (5x5 matrix)

- Extracts high level Plasma quantities such as normal wave vector, wave ellipticity estimator, wave degree of polarization, z-component of the normalized Poynting vector, phase velocity estimator,...

- Provides continuous waveforms up to 4096S/s on demand (Burst modes)

# Low Frequency Receiver (LFR)

- 5 **E-field** and 3 **B-field** waves from ~0Hz to 10kHz -> 8 inputs
- less than 2W
- ~3-4kb/s average telemetry
- 240x160mm form factor
- Radiation hardened and fault tolerant
- -30°C : +75°C temperature range

<div>
<center>
<img src="resources/LFR_Block_Diagram.jpg" width="80%" align="center">
</center>
</div>

<div>
<center>
<img src="resources/LFR-PFM-Pict.png" width="80%" align="center">
</center>
</div>

# Sampling you said... what about aliasing? 
## Strategy 

- 10KHz signal bandwidth 
- limited telemetry -> sampling frequency close to 2 x 10kHz
- Expected resolution ~16 bits -> ~96dB dynamic 
- Full analog filtering is near impossible or stupid :)
- OPAMP power consumption is ~2.5mW

# Sampling you said... what about aliasing? 
## Analog front-end 
<div>
<center>
<img src="resources/LFR_first_stage_bands.png" width="70%" align="center">
</center>
</div>


# Sampling you said... what about aliasing? 
## Analog front-end
<div>
<center>
<img src="resources/LFR_front_end.png" width="80%" align="center">
</center>
</div>


# Sampling you said... what about aliasing? 
## Analog front-end 
<div>
<center>
<img src="resources/LFR_front_end_TF.png" width="80%" align="center">
</center>
</div>


# Sampling you said... what about aliasing?
## Digital filters \o/
<div>
<center>
<img src="resources/LFR_first_stage_IIR_bands.png" width="70%" align="center">
</center>
</div>

# Digital filters
## general considerations

- usual form $$y(n) = \sum_{i=0}^{M} b_i x(n-i) - \sum_{j=0}^{N} a_j y(n-j)$$

# Digital filters
## general considerations
<div>
<center>
<img src="resources/DFI_IIR.png" width="70%" align="center">
</center>
</div>

# Digital filters
## general considerations

**Space FPGAs (2008-2014):**

- Few or no DSP blocks (0 to 120)
- Few RAM blocks (up to 540kb)
- Slow (LEON3 @ 25MHz max)


<div>
<center>
<img src="resources/space_components.png" width="70%" align="center">
</center>
</div>

# Ok, digital filter(s) but which one(s)?

- **FIR** -> too many operations and memory
- **Multi-rate FIR** -> still too many memory
- **CIC** -> benefits come at high decimation rate, sin(X)/X like TF
- **IIR** -> needs careful design

<div>
<center>
<img src="resources/LFR_processing_flow.png" width="80%" align="center">
</center>
</div>

# 98 304Hz to 24 576Hz filter

- 5 stages Direct Form I
<div>
<center>
<img src="resources/IIR_TF.png" width="80%" align="center">
</center>
</div>

# 98 304Hz to 24 576Hz filter
## RTL implementation considerations

- Time over space tradeoff
- One **MAC** capable **ALU** to filter them all
- Use one **RAM** block to store samples instead of R-Cells
- Design really close to what a simple CPU could be


# 98 304Hz to 24 576Hz filter
## RTL implementation flow, don't try to be smart

1. Matlab or SciPy.signal
2. Naive implementation C/C++/Python...
3. Fixed point conversion/optimization
4. RTL implementation with tests

observations:
* impulse response is easy to compute...
* one can read/write files from VHDL simulation...

# 98 304Hz to 24 576Hz filter
## Implementation pitfalls

- No RAM clearing breaks causality!
- Filter stage structures matters! (Direct Form I/II...)
- Last clock cycles can be hard to save


<div>
<center>
<img src="resources/bugs_meme.jpeg" width="60%" align="center">
</center>
</div>

# Offset bug

Sometime LFR starts with a random offset or an oscillation on some channels

1. reproduce on instrument
2. investigate 
3. reproduce on simulation
4. implement non regression test
5. fix the bug
6. buy new FPGAs :(


<div>
<center>
<img src="resources/LFR_OFFSET_BUG.png" width="80%" align="center">
</center>
</div>

<div>
<center>
<img src="resources/LFR_OFFSET_NOBUG.png" width="80%" align="center">
</center>
</div>

# Offset bug

- When initialized with random content a fixed point recursive filter may start in an "unstable" state 
- Doesn't affect floating point implementation of recursive filters
- Always reset filters with 0!

# Done with bugs?

<div>
<center>
<img src="resources/BUG_MACHINE.jpeg" width="40%" align="center">
</center>
</div>

# ADC bug -20°C

<div>
<center>
<img src="resources/ADC_BUG_WF.png" width="80%" align="center">
</center>
</div>

# ADC bug

<div>
<center>
<img src="resources/ADC_BUG_SPECT.png" width="90%" align="center">
</center>
</div>

<div>
<center>
<img src="resources/PCB_LFR.png" width="80%" align="center">
</center>
</div>

# ADC bug 
## Layout equivalent model
<div>
<center>
<img src="resources/CLK_EQ_DIAG.png" width="80%" align="center">
</center>
</div>

# ADC bug
## QUCS simulation
<div>
<center>
<img src="resources/QUCS_CLK.png" width="80%" align="center">
</center>
</div>

# ADC bug
## Conclusion

- Mostly related to PCB
- Thermal models shows that LFR won't be colder than 10°C
- Components involved in the model don't changes with radiations
- That was really a mistake to connect 8 ADCs on the same CLK

<div>
<center>
<img src="resources/ignore-so-hard.jpg" width="90%" align="center">
</center>
</div>

<div>
<center>
<img src="resources/automated-tests.jpg" width="60%" align="center">
</center>
</div>

# Reminder, LFR SOC
<div>
<center>
<img src="resources/LFR_SOC.png" width="60%" align="center">
</center>
</div>

# Existing solutions?
## Grmon?

- Closed source -> not extensible
- Expensive
- Can't be scripted with Python
- Focuses on Gaisler products

# SocExplorer 
## a flexible framework to test them all

- Focuses on SOC control/exploration
- Allows from IP unit testing to system level performances measurements
- From manual registers investigation to automated head-less tests batch
- Gives a remote access to the whole SOC memory from your computer
- Heavily extensible with plugins
- Script with Python -> can virtually do anything with python
- Used for all LFR tests, from hardware to software

<div>
<center>
<img src="resources/SocExplorer1.png" width="80%" align="center">
</center>
</div>

<div>
<center>
<img src="resources/SocExplorer2.png" width="80%" align="center">
</center>
</div>

<div>
<center>
<img src="resources/SocExplorer3.png" width="100%" align="center">
</center>
</div>

# SocExplorer 
## LFR 2nd gen python test framework

- Uses a generic launcher script
- Inspired by C++ unit tests frameworks (GTests or QTests)
- Allows to build tests from simple boot test to TF
- Allows high repeatability
- Intensive use of Python serialization features

# SocExplorer 
## Setup example

<div>
<center>
<img src="resources/SetupSynchro.png" width="100%" align="center">
</center>
</div>

# Lessons learned

- What is not tested doesn't work
- Tools are the key to success
- Prefer open source solutions to proprietary ones

# What next?

- Both code and HW are **open source**!
- Next generation could rely on tinny **CPU arrays** -> **ASIC**? **RISC-V**?
- Onboard **machine learning** for event detection and classification
- A lot of **reusable blocks** for ground equipment
- Open source flexible testbench under development
- Analyzers knowledge used for next generation digital search-coils

# Contributors:
- Thomas Chust
- Paul Leroy
- Martin Morlot
- Yannic Simon
- Clotilde Laigle
- Pedro Luiz Coelho Rodrigues
- Gerald Saule
- Jean-Christophe Pellion
- Bruno Katra
- Veronique Bouzid